In [1]:
import pyrosetta
import pyrosetta.distributed.dask
import pyrosetta.distributed.io as io
import pyrosetta.distributed.packed_pose as packed_pose
import pyrosetta.distributed.tasks.rosetta_scripts as rosetta_scripts
import pyrosetta.distributed.tasks.score as score
from dask.distributed import Client, LocalCluster
import logging



In [2]:
my_flags = "-extra_res_fa ATP.am1-bcc.fa.params 295.am1-bcc.fa.params -score:extra_improper_file Temp_ATP/ATP.am1-bcc.fa.tors"

pyrosetta.distributed.dask.init_notebook(my_flags)

core.init: Checking for fconfig files in pwd and ./rosetta/flags
core.init: Rosetta version: 2018.51.post.dev+654.v2019.01dev60566.dbc838b6ae6 dbc838b6ae620b1293476b1bd4366ffc2facc5b5 ssh://git@github.com/RosettaCommons/main 2019-01-03T10:31:13
core.init: command: PyRosetta -ex1 -ex2aro -extra_res_fa ATP.am1-bcc.fa.params 295.am1-bcc.fa.params -score:extra_improper_file Temp_ATP/ATP.am1-bcc.fa.tors -database /Users/Loulou/anaconda3/envs/pyrosetta-code-school/lib/python3.6/site-packages/pyrosetta/database
core.init: 'RNG device' seed mode, using '/dev/urandom', seed=-1174091748 seed_offset=0 real_seed=-1174091748
core.init.random: RandomGenerator:init: Normal mode, seed=-1174091748 RG_type=mt19937


In [3]:
pose_start = score.ScorePoseTask(weights="ref2015_cart")(io.pose_from_file("CDK9_ATP_3BLQ_relaxed_chX.pdb"))

core.chemical.GlobalResidueTypeSet: Finished initializing fa_standard residue type set.  Created 638 residue types
core.chemical.GlobalResidueTypeSet: Total time to initialize 1.08206 seconds.
core.import_pose.import_pose: File 'CDK9_ATP_3BLQ_relaxed_chX.pdb' automatically determined to be of type PDB
core.io.pose_from_sfr.PoseFromSFRBuilder: [ WARNING ] discarding 14 atoms at position 296 in file CDK9_ATP_3BLQ_relaxed_chX.pdb. Best match rsd_type:  ATP
core.scoring.etable: Starting energy table calculation
core.scoring.etable: smooth_etable: changing atr/rep split to bottom of energy well
core.scoring.etable: smooth_etable: spline smoothing lj etables (maxdis = 6)
core.scoring.etable: smooth_etable: spline smoothing solvation etables (max_dis = 6)
core.scoring.etable: Finished calculating energy tables.
basic.io.database: Database file opened: scoring/score_functions/hbonds/ref2015_params/HBPoly1D.csv
basic.io.database: Database file opened: scoring/score_functions/hbonds/ref2015_para

core.scoring.CartesianBondedEnergy: Adding undefined angle ATP: C5',C4',C3' to DB with theta0 = 1.96796 , Ktheta = 80
core.scoring.CartesianBondedEnergy: Adding undefined angle ATP: C5',C4',H2 to DB with theta0 = 1.85117 , Ktheta = 80
core.scoring.CartesianBondedEnergy: Adding undefined angle ATP: C3',C4',H2 to DB with theta0 = 1.89878 , Ktheta = 80
core.scoring.CartesianBondedEnergy: Adding undefined angle ATP: C4',O4',C1' to DB with theta0 = 1.92162 , Ktheta = 80
core.scoring.CartesianBondedEnergy: Adding undefined angle ATP: O4',C1',N9 to DB with theta0 = 1.91346 , Ktheta = 80
core.scoring.CartesianBondedEnergy: Adding undefined angle ATP: O4',C1',C2' to DB with theta0 = 1.85709 , Ktheta = 80
core.scoring.CartesianBondedEnergy: Adding undefined angle ATP: O4',C1',H7 to DB with theta0 = 1.9487 , Ktheta = 80
core.scoring.CartesianBondedEnergy: Adding undefined angle ATP: N9,C1',C2' to DB with theta0 = 1.92459 , Ktheta = 80
core.scoring.CartesianBondedEnergy: Adding undefined angle ATP

core.scoring.CartesianBondedEnergy: Adding undefined length ATP: C1',N9, to DB with d0 = 1.48422 , Kd = 300
core.scoring.CartesianBondedEnergy: Adding undefined length ATP: C1',C2', to DB with d0 = 1.52803 , Kd = 300
core.scoring.CartesianBondedEnergy: Adding undefined length ATP: C1',H7, to DB with d0 = 1.07029 , Kd = 300
core.scoring.CartesianBondedEnergy: Adding undefined length ATP: N9,C8, to DB with d0 = 1.34331 , Kd = 300
core.scoring.CartesianBondedEnergy: Adding undefined length ATP: N9,C4, to DB with d0 = 1.3368 , Kd = 300
core.scoring.CartesianBondedEnergy: Adding undefined length ATP: C8,N7, to DB with d0 = 1.35621 , Kd = 300
core.scoring.CartesianBondedEnergy: Adding undefined length ATP: C8,H11, to DB with d0 = 1.03198 , Kd = 300
core.scoring.CartesianBondedEnergy: Adding undefined length ATP: N7,C5, to DB with d0 = 1.3413 , Kd = 300
core.scoring.CartesianBondedEnergy: Adding undefined length ATP: C5,C4, to DB with d0 = 1.47103 , Kd = 300
core.scoring.CartesianBondedEnergy

In [4]:
def run_fast_design(pose_start):
    
    pose_start = packed_pose.to_pose(pose_start)
    
    scorefxn_cart = pyrosetta.create_score_function("ref2015_cart")
    atp_res = pyrosetta.rosetta.core.select.residue_selector.ChainSelector("X")
    
    # Selector for repacking NOT ligand + small shell
    neighboor = pyrosetta.rosetta.core.select.residue_selector.NeighborhoodResidueSelector(selector=atp_res, 
                                                                                           distance=8,
                                                                                           include_focus_in_subset=False)
    
    # Selector for minimization Ligand + broad shell
    neighboor2 = pyrosetta.rosetta.core.select.residue_selector.NeighborhoodResidueSelector(selector=atp_res, 
                                                                                           distance=16,
                                                                                           include_focus_in_subset=True)
    
    not_neighboor = pyrosetta.rosetta.core.select.residue_selector.NotResidueSelector(neighboor)
    not_neighboor2 = pyrosetta.rosetta.core.select.residue_selector.NotResidueSelector(neighboor2)

    # Initiate TaskFactory
    tf = pyrosetta.rosetta.core.pack.task.TaskFactory()

    tf.push_back(pyrosetta.rosetta.core.pack.task.operation.InitializeFromCommandline())
    tf.push_back(pyrosetta.rosetta.core.pack.task.operation.IncludeCurrent())

    # Disable design outside design shell
    tf.push_back(
        pyrosetta.rosetta.core.pack.task.operation.OperateOnResidueSubset(
            pyrosetta.rosetta.core.pack.task.operation.PreventRepackingRLT(), not_neighboor)
    )

    
    # Remove Cys from design
    aa_to_design = pyrosetta.rosetta.core.pack.task.operation.RestrictAbsentCanonicalAASRLT()
    aa_to_design.aas_to_keep("ADEFGHIKLMNPQRSTVWY")
    tf.push_back(pyrosetta.rosetta.core.pack.task.operation.OperateOnResidueSubset(aa_to_design, neighboor))
                 
                 
    # Convert task factory into PackerTask just to view 
    #packer_task = tf.create_task_and_apply_taskoperations(pose_start)
    
    
    # Set MoveMapFactory
    
    # Set up a MoveMapFactory
    mmf = pyrosetta.rosetta.core.select.movemap.MoveMapFactory()
    mmf.all_bb(setting=True)  # Set to true if needed
    mmf.all_bondangles(setting=True)
    mmf.all_bondlengths(setting=True)
    mmf.all_chi(setting=True)
    mmf.all_jumps(setting=True)
    mmf.set_cartesian(setting=True)

# Set movemap actions to turn on or off certain torsions, overriding the above defaults
    enable = pyrosetta.rosetta.core.select.movemap.move_map_action.mm_enable
    disable = pyrosetta.rosetta.core.select.movemap.move_map_action.mm_disable

# Set custom minimizable torsions

    mmf.add_bondangles_action(action=disable, selector=neighboor2)
    mmf.add_bondlengths_action(action=disable, selector=neighboor2)
    mmf.add_chi_action(action=disable, selector=neighboor2)

    
    
    

    # Set up FastDesign

    fast_design = pyrosetta.rosetta.protocols.denovo_design.movers.FastDesign(scorefxn_in=scorefxn_cart, 
                                                                              standard_repeats=5)
    fast_design.set_task_factory(tf)
    fast_design.set_movemap_factory(mmf)
    fast_design.set_scorefxn(scorefxn_cart)
    fast_design.set_cartesian(True)
    
    # Apply FastDesign
    fast_design.apply(pose_start)
    
    return score.ScorePoseTask(weights="ref2015_cart")(packed_pose.to_packed(pose_start))



In [5]:
cluster = LocalCluster(n_workers=4, threads_per_worker=1)
client = Client(cluster)
 


/Users/Loulou/anaconda3/envs/pyrosetta-code-school/lib/python3.6/site-packages/distributed/bokeh/core.py:57: UserWarning: 
Port 8787 is already in use. 
Perhaps you already have a cluster running?
Hosting the diagnostics dashboard on a random port instead.
  warnings.warn('\n' + msg)


In [6]:
client


Client Scheduler: tcp://127.0.0.1:55093 Dashboard: http://127.0.0.1:55094/status,Cluster Workers: 4 Cores: 4 Memory: 8.59 GB


In [ ]:
client.run(pyrosetta.distributed.dask.init_notebook, my_flags)

futures = [client.submit(run_fast_design, pose_start) for i in range(100)]
logging.info("Now performing refinement on each point mutant!")
results = [future.result() for future in futures]

client.close()
cluster.close()

/Users/Loulou/anaconda3/envs/pyrosetta-code-school/lib/python3.6/site-packages/distributed/worker.py:2791: UserWarning: Large object of size 1.82 MB detected in task graph: 
  (<pyrosetta.distributed.packed_pose.core.PackedPos ...  0x111959588>,)
Consider scattering large objects ahead of time
with client.scatter to reduce scheduler burden and 
keep data on workers

    future = client.submit(func, big_data)    # bad

    big_future = client.scatter(big_data)     # good
    future = client.submit(func, big_future)  # good
  % (format_bytes(len(b)), s))
